# Лабораторная работа 7
## Знакомство с Pytorch


1. Базовые операции

    1.1.  Сгенерировать искусcтвенный набор данных, состоящий из 3 гауссовых облаков и визуализировать его (_Используйте torch.distributions_)
    
    1.2.  Выполните нормировку созданного набора данных
    
2. Реализация собственного набора данных с помощью `DatasetIterable`
    
    * Используйте документацию: https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset
    * Подразумевается, что данные будут генерироваться внутри метода `__iter__`. Данный метод будет возвращать кортеж (data_sample, label). 
    
    Например: `(torch.Tensor([0.0, 0.0]), 1)  # элемент набора данных (2 признака) и метка класса (класс №1)`
    * В конструкторке класса должны быть созданы объекты распределений (`torch.distributions`, как в предыдущем задании)

3. Примените предобученную нейронную сеть к вашему набору данных, используя `pytorch-lightning`

    * Можете использовать любую нейронную сеть, доступную в `pytorch`
    * Приветствуется применение логгирования в любую из систем: comet.ml, weightsAndBiases, Tensorboard (вы можете использовать [Logger](https://pytorch-lightning.readthedocs.io/en/latest/common/loggers.html) из pytorch-lightning)
    * Также добавьте в pytorch_lightning.LightningModule вычисление какой-либо метрики, например F1-score (допускается использовать для этого scikit-learn)
    * Сохраните лучшую модель
    
4. (Опционально) Используя `pytorch`, реализуйте любой алгоритм машинного обучения (дискриминантный анализ, k-means, перцептрон, и т.д.) и проверьте его на любом наборе данных. Пример реализации логистической регрессии:

```python
class LogisticRegression(nn.Module):
    def __init__(self, dim, lr=torch.scalar_tensor(0.01)):
        super(LogisticRegression, self).__init__()

        self.w = torch.zeros(dim, 1, dtype=torch.float)
        self.b = torch.scalar_tensor(0)
        self.grads = {
            "dw": torch.zeros(dim, 1, dtype=torch.float),
            "db": torch.scalar_tensor(0)
        }
        self.lr = lr

    def forward(self, x):
        z = torch.mm(self.w.T, x)  # mm == Matrix Multiplication
        a = self.sigmoid(z)
        return a

    def sigmoid(self, z):
        return 1/(1 + torch.exp(-z))

    def backward(self, x, y_hat, y):
        self.grads["dw"] = (1/x.shape[1]) * torch.mm(x, (y_hat - y).T)
        self.grads["db"] = (1/x.shape[1]) * torch.sum(y_hat - y)
    
    def optimize(self):
        self.w = self.w - self.lr * self.grads["dw"]
        self.b = self.b - self.lr * self.grads["db"]
```

In [1]:
# INIT
import math
import copy
import torch
import torchvision
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt


import torch.distributions as tdist
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from torch.distributions import normal, Normal, MultivariateNormal, LKJCholesky, CumulativeDistributionTransform, Normal, TransformedDistribution, Dirichlet

from sklearn.utils import shuffle
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

%matplotlib inline

### Tools

In [3]:
class MyIterableDataset(torch.utils.data.IterableDataset):
     def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        
        assert end > start, "this example code only works with end >= start"
            self.start = start
            self.end = end
        
        def __iter__(self):
            worker_info = torch.utils.data.get_worker_info()
...         if worker_info is None:  # single-process data loading, return the full iterator
    
    iter_start = self.start
...             iter_end = self.end
...         else:  # in a worker process
...             # split workload
...             per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
...             worker_id = worker_info.id
...             iter_start = self.start + worker_id * per_worker
...             iter_end = min(iter_start + per_worker, self.end)
...         return iter(range(iter_start, iter_end))
...
>>> # should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
>>> ds = MyIterableDataset(start=3, end=7)

IndentationError: unexpected indent (2275813290.py, line 6)

### 1.1. Сгенерировать искусcтвенный набор данных, состоящий из 3 гауссовых облаков и визуализировать его (Используйте torch.distributions)

In [ ]:
def main (N,k) :
    # Генерируем облака согласно заданию штук
    clouds = []
    for i in range(N):
        clouds.append(noise + np.array([k*i,k*i]))
    x = np.vstack(clouds)
    
    y = []
    for i in range(N):
        y+=[i] * clouds[i].shape[0]  # объединения списков; lists concatenation
    
    data, labels = shuffle(x, y)
    
    return clouds

In [ ]:
z = 0
N = 3
M = 797
R1 = 0.743 
R2 = 0.752

m = Dirichlet(torch.tensor([R1, R2]))
noise = m.sample([M,2])

#m = normal.Normal(R1,R2)
#noise = m.sample([M,2])

for i in range(100, 200, 1):
    clouds = main(N,i/100)

for i in range(N):
    plt.scatter(clouds[i][:, 0], clouds[i][:, 1], c='red', marker='o', edgecolors = 'black', alpha = 0.6)

### 1.2. Выполните нормировку созданного набора данных

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((R1, R2), (R1))])
 
trainset = clouds 
#torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2, shuffle=True, num_workers=2)
dataiter = iter(trainloader)
data, labels = dataiter.next()

In [ ]:
data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=33)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))

ax1.scatter(X_train[:,0][y_train == 0], X_train[:,1][y_train == 0], c='red', marker='o', edgecolors = 'black', alpha = 0.6)
ax1.scatter(X_train[:,0][y_train == 1], X_train[:,1][y_train == 1], c='green', marker = 'o', edgecolors = 'black', alpha = 0.6)
ax1.scatter(X_train[:,0][y_train == 2], X_train[:,1][y_train == 2], c='blue', marker = 'o', edgecolors = 'black', alpha = 0.6)
ax1.set_title('Train', fontsize=20)

ax2.scatter(X_test[:,0][y_test == 0], X_test[:,1][y_test == 0], c='red', marker='o', edgecolors = 'black', alpha = 0.6)
ax2.scatter(X_test[:,0][y_test == 1], X_test[:,1][y_test == 1], c='green', marker = 'o', edgecolors = 'black', alpha = 0.6)
ax2.scatter(X_test[:,0][y_test == 2], X_test[:,1][y_test == 2], c='blue', marker = 'o', edgecolors = 'black', alpha = 0.6)
ax2.set_title('Test', fontsize=20);